### 2. ekonlpy를 사용해 token, ngram을 구한다.

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime

# df_1 = pd.read_csv('../datasets/1_article_raw.csv')
# df_2 = pd.read_csv('../datasets/1_pdf_2005_2017.csv')
# df_3 = pd.read_csv('../datasets/1_minutes_with_label.csv')

# del df_1['Unnamed: 0']
# del df_1['idx']
# del df_1['media']

# del df_2['Unnamed: 0']
# del df_2['title']
# del df_2['company']
# del df_2['link']

# del df_3['Unnamed: 0']
# del df_3['title']
# del df_3['ngram']
# del df_3['label']

# 채권분석 날짜 형변환
# df_2['date'] = df_2['date'].apply(lambda x: datetime.strptime(x, '%y.%m.%d').strftime('%Y-%m-%d'))

# # 뉴스 & 채권분석리포트 & 금통위 의사록 합치기
news_report_mpb = pd.concat([df_1, df_2, df_3])

# # 결측값있는 행 제거
news_report_mpb = news_report_mpb.dropna(axis=0)
news_report_mpb.to_csv('../datasets/2_news_report_mpb.csv', index=False)

In [ ]:
import pandas as pd
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK

mpck = MPCK()

pdf = pd.read_csv('../datasets/2_news_report_mpb.csv', index_col=0)

print(len(pdf))

pdf['ngram'] = ''

for idx , sent in zip(pdf.index, pdf['content']):
    tokens = []
    if (idx % 100) == 0:
            print('{}번째 끝'.format(idx)) 
    for s in sent.split('.'):
    
        # mpck.tokenize를 사용해 논문에서 사용하는 5개의 품사만 추출해 토큰화 한다. => uni-gram
        tokens += mpck.tokenize(s)
        # pdf.loc[idx, 'token'] = tokens
    print(len(tokens))
    # mpck.ngramize를 사용해 bi-gram ~ 5-gram까지 ngram을 나눈다.
    ngrams = mpck.ngramize(tokens)
    # pdf.loc[idx, 'ngrams'] = ngrams

    # 'token'컬럼과 '2-5gram'을 합쳐서 '1~5'그램을 만든다.
    pdf['ngram'][idx] = tokens+ngrams

pdf.to_csv('../datasets/2_nrm_ngram.csv', index=False)
pdf.head()

### 2-1. 날짜를 기준으로 리포트와 콜금리의 라벨을 병합한다.

In [ ]:
import pandas as pd
from datetime import datetime

nrm  = pd.read_csv('../datasets/2_nrm_ngram.csv')
callrate = pd.read_csv('../datasets/2_callrate_preprocessing.csv')

# call금리의 date컬럼의 형식을 바꾸기(YYYY-MM-DD)
callrate['date'] = callrate['date'].apply(lambda x: datetime.strptime(x, '%Y.%m.%d').strftime('%Y-%m-%d'))

# 콜금리 컬럼을 date와 label만 사용해서 merge하기
nrt_call = pd.merge(nrm, callrate[['date', 'label']], on='date')

# report와 call 금리 병합 데이터 저장하기
nrt_call.to_csv('../datasets/2_nrm_callrate_merge.csv', index=False)